In [1]:
# Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Import classes from Class.py
from Class import *
from Data_handling import *

In [ ]:
## DATA INDEX

Data_df

,Name,Size,Content
0,Dem,"(3600, 17)",Demand for each load for each hour of the inve...
1,Uti,"(17, 1)",Utility for each load for one hour
2,Load_Z,"(2, 17)",Zone of each load
3,Gen_E_OpCost,"(16, 1)",Operationnal cost of each existing generator
4,Gen_N_OpCost,"(16,)",Operationnal cost of each new generator
5,Gen_E_Cap,"(16, 1)",Maximum capacity for existing units
6,Gen_N_MaxInvCap,"(16, 1)",Maximum capacity investment of each new generator
7,Gen_N_InvCost,"(16, 1)",Unit Investment cost of each new generator
8,Gen_E_Tech,"(16, 1)",Technology of each existing generator
9,Gen_N_Tech,"(16, 1)",Technology of each new generator


In [3]:
## PARAMETERS DEFINITION

# Time
H = 24          # Hours in a day
D = 5           # Typical days in a year
Y = 30          # Years of the investment timeline
N = H*D*Y       # Number of hours in the investment timeline    

# Number of loads and generators
N_dem = len(Dem[0,:])       # Number of loads
N_gen_E = len(Gen_E_OpCost)   # Number of existing generators
N_gen_N = len(Gen_N_OpCost)   # Number of new generators
N_zone = len(Trans_Z_Connected_To_Z)     # Number of zones
N_line = len(Trans_Line_From_Z)   # Number of transmission lines

# Hyperparameters
B = 1000000000   # Budget for the investment problem
R = 73 # Conversion rate


In [4]:
## CREATE THE PARAMETERS AND DATA OBJECTS
ParametersObj = Parameters(H, D, Y, N, N_dem, N_gen_E, N_gen_N, N_zone, N_line, B, R)
DataObj = InputData(Dem, Uti, Load_Z, Gen_E_OpCost, Gen_N_OpCost, Gen_N_MaxInvCap, Gen_E_Cap, Gen_N_InvCost, Gen_E_Tech, Gen_N_Tech, Gen_E_Z, Gen_N_Z, Gen_E_OpCap, Gen_N_OpCap, Trans_React, Trans_Cap, Trans_Line_From_Z, Trans_Line_To_Z, Trans_Z_Connected_To_Z)

# Model 1: Sequential optimization of Dispatch problem and Investment problem

### 1) Market Clearing

In [5]:
# Run the Market Clearing Problem
MarketClearing1 = MarketClearingModel1(ParametersObj, DataObj)

Set parameter Username


Academic license - for non-commercial use only - expires 2025-09-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 140400 rows, 126000 columns and 270000 nonzeros
Model fingerprint: 0x4c1471a4
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+01, 1e+03]
  Bounds range     [1e+03, 1e+03]
  RHS range        [4e-01, 1e+03]
Presolve removed 134239 rows and 109069 columns
Presolve time: 0.75s
Presolved: 6161 rows, 16931 columns, 19670 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.739e+03
 Factor NZ  : 8.900e+03 (roughly 10 MB of memory)
 Factor Ops : 1.438e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       

In [6]:
MarketClearing1.res.df.head(10)

,Hour,Load Zone 0,Existing generators Zone 0,Price Zone 0,Load Zone 1,Existing generators Zone 1,Price Zone 1,Power flow line 1,Power flow line 2
0,1,1032.317322,882.317322,70.0,752.303582,902.303582,-0.0,-150.000000,150.000000
1,2,957.988480,807.988480,70.0,698.136270,848.136270,-0.0,-150.000000,150.000000
2,3,912.112663,762.112663,70.0,664.704165,814.704165,-0.0,-150.000000,150.000000
3,4,906.650831,756.650831,70.0,660.723842,810.723842,-0.0,-150.000000,150.000000
4,5,924.897360,774.897360,70.0,671.693908,821.693908,30.0,-150.000000,150.000000
5,6,939.461824,972.512328,70.0,625.175318,592.124814,70.0,33.050504,-33.050504
6,7,1100.000000,1250.000000,75.0,591.679050,441.679050,75.0,150.000000,-150.000000
7,8,1100.000000,1250.000000,75.0,527.349585,377.349585,125.0,150.000000,-150.000000
8,9,1100.000000,1250.000000,125.0,510.104485,360.104485,125.0,150.000000,-150.000000
9,10,1100.000000,1250.000000,125.0,456.282895,306.282895,125.0,150.000000,-150.000000


### 2) Investment Problem

In [7]:
# Run the investmentmodel
InvestmentPB1 = InvestmentModel1(ParametersObj, DataObj, MarketClearing1.res.DA_price)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 57857 rows, 57616 columns and 111932 nonzeros
Model fingerprint: 0x50561b0a
Coefficient statistics:
  Matrix range     [4e-04, 1e+06]
  Objective range  [1e+03, 1e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+09]
Presolve removed 57856 rows and 57605 columns
Presolve time: 0.18s
Presolved: 1 rows, 11 columns, 11 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3734490e+10   4.096000e+05   0.000000e+00      0s
       1    1.8264654e+10   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.32 seconds (0.06 work units)
Optimal objective  1.826465359e+10
Objective value:  18264653589.580414


In [8]:
InvestmentPB1.res.df

,Technology,Invested capacity (MW)
0,Coal,0.0
1,Coal,0.0
2,Coal,0.0
3,Gas,0.0
4,Coal,0.0
5,Wind,200.0
6,Wind,200.0
7,PV,200.0
8,Gas,0.0
9,Coal,0.0


# Model 2: Integrated Bi-level optimization of dispatch problem and investment problem using KKTs